In [1]:
import rasterio
import geopandas as gpd
import pandas as pd
import numpy as np
from pyproj import Transformer
from datetime import datetime
import json
import re
import os
import zipfile
from datetime import datetime
from fuzzywuzzy import process
import logging
import warnings
import shutil
import hvplot.pandas
from shapely.geometry import Point, Polygon, box
import panel as pn
from bokeh.palettes import Inferno256

#### Preprocessing data


##### Filtering by specific postal code for spatial map plotting (Bukit Purmei)

In [1]:
# Exporting data for blocks of interest and control blocks
import geopandas as gpd

geojson_path = "C:\\LocalOneDrive\\Documents\\Desktop\\MTI\\UHI-Project\\MSE-ES-UHI\\Data\\ADDRPT.geojson"
postal_code_112 = "090112"
postal_code_114 = "090114"
postal_code_113 = "090113"
postal_code_115 = "090115"

# Load the GeoJSON file
gdf = gpd.read_file(geojson_path)

# Function to retrieve coordinates by postal code
def get_coordinates_by_postal_code(postal_code):
    # Filter GeoDataFrame for the given postal code
    filtered_gdf = gdf[gdf['POSTAL_CODE'] == postal_code]
    if not filtered_gdf.empty:
        # Extract coordinates
        point = filtered_gdf.iloc[0].geometry
        return point.x, point.y
    else:
        return None, None

longitude_112, latitude_112 = get_coordinates_by_postal_code(postal_code_112)
longitude_114, latitude_114 = get_coordinates_by_postal_code(postal_code_114)
longitude_113, latitude_113 = get_coordinates_by_postal_code(postal_code_113)
longitude_115, latitude_115 = get_coordinates_by_postal_code(postal_code_115)

if longitude_112 and latitude_112 and longitude_114 and latitude_114:
    print(f'Coordinates for postal code {postal_code_112}: Longitude {longitude_112}, Latitude {latitude_112}')
    print(f'Coordinates for postal code {postal_code_114}: Longitude {longitude_114}, Latitude {latitude_114}')
    print(f'Coordinates for postal code {postal_code_113}: Longitude {longitude_113}, Latitude {latitude_113}')
    print(f'Coordinates for postal code {postal_code_115}: Longitude {longitude_115}, Latitude {latitude_115}')
else:
    print('Postal code not found.')

Coordinates for postal code 090112: Longitude 103.82593292805574, Latitude 1.2745285256209595
Coordinates for postal code 090114: Longitude 103.82588719010951, Latitude 1.2750718182249274
Coordinates for postal code 090113: Longitude 103.82693226761857, Latitude 1.2747701258018154
Coordinates for postal code 090115: Longitude 103.82695018030107, Latitude 1.2753520132629723


In [2]:
# Finding the central postal code for Blocks 112, 114, 113 and 115
if longitude_112 and latitude_112 and longitude_114 and latitude_114:
    # Calculate the average coordinates
    avg_longitude = (longitude_112 + longitude_114 + longitude_113 + longitude_115) / 4
    avg_latitude = (latitude_112 + latitude_114 + latitude_113 + latitude_115) / 4
    print(f'Central coordinates:')
    print(f'Longitude: {avg_longitude}, Latitude: {avg_latitude}')
else:
    print('Postal code not found.')

Central coordinates:
Longitude: 103.82642564152123, Latitude: 1.2749306207276687


In [3]:
# Converting x and y to coordinates for latitude/longitude

global filtered_df

def preprocessing(file_path):   
    global filtered_df
    
    # Open your GeoTIFF file
    with rasterio.open(file_path) as src:
        array = src.read()
        transform = src.transform
        src_crs = src.crs  # Source CRS
        dest_crs = 'EPSG:4326'  # WGS 84

        # Create a transformer object to convert from src_crs to dest_crs
        transformer = Transformer.from_crs(src_crs, dest_crs, always_xy=True)

        # Get arrays of column and row indices
        cols, rows = np.meshgrid(np.arange(array.shape[2]), np.arange(array.shape[1]))
        
        # Convert meshgrid arrays to coordinate arrays using rasterio's method, which are 2D
        xs, ys = rasterio.transform.xy(transform, rows, cols, offset='center')
        
        # Flatten the coordinate arrays to pass to transform function
        lon, lat = transformer.transform(np.array(xs).flatten(), np.array(ys).flatten())

        # Create DataFrame and convert to GeoDataFrame
        df = pd.DataFrame({'Longitude': lon, 'Latitude': lat})
        for i, band in enumerate(src.read(masked=True)):
            df[src.descriptions[i]] = band.flatten()

        # # Convert 'SR_QA_AEROSOL' to integer for bitwise operation
        # df['SR_QA_AEROSOL'] = df['SR_QA_AEROSOL'].astype(int)

        # # Filter out pixels with valid aerosol retrieval and high aerosol level
        # # Assuming 'SR_QA_AEROSOL' is the name of the QA aerosol band in the data
        # valid_aerosol = (df['SR_QA_AEROSOL'] & 2) == 2  # Bit 1 must be set for valid retrieval
        # high_aerosol = (df['SR_QA_AEROSOL'] & 192) == 192  # Bits 6-7 must be set to 11 for high aerosol
        # filter_mask = valid_aerosol & high_aerosol
        # df_filtered = df[-filter_mask]

        df_filtered = df
        
        # Scale and offset specific bands
        df_filtered['ST_B10_Celsius'] = df_filtered['ST_B10'] * 0.00341802 + 149 - 273.15
        df_filtered = df_filtered[df_filtered['ST_B10_Celsius'] >= 20]  # Drop rows below 20 degrees Celsius
        
        bands_to_scale = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']
        for band in bands_to_scale:
            df_filtered[f"{band}_Scaled"] = df_filtered[band] * 2.75e-05 - 0.2

        additional_scales = {
            'ST_ATRAN': 0.0001, 'ST_CDIST': 0.01, 'ST_DRAD': 0.001, 
            'ST_EMIS': 0.0001, 'ST_EMSD': 0.0001, 'ST_QA': 0.01, 
            'ST_TRAD': 0.001, 'ST_URAD': 0.001
        }

        for band, scale in additional_scales.items():
            df_filtered[f"{band}_Scaled"] = df_filtered[band] * scale

        gdf = gpd.GeoDataFrame(df_filtered, geometry=gpd.points_from_xy(df_filtered.Longitude, df_filtered.Latitude))
        gdf.set_crs('EPSG:4326', inplace=True)  # Ensure the CRS is set to WGS 84

        print("Total number of valid pixels: " + str(len(gdf)))
        print(df[['Latitude', 'Longitude']].head())

        # Define your point of interest and buffer distance in meters
        poi = Point(avg_longitude, avg_latitude)
        desired_radius = 200
        buffer = poi.buffer(desired_radius / 111320)  # Convert meters to degrees approximately

        # Filter points within the buffer
        filtered_gdf = gdf[gdf.geometry.within(buffer)]

        # Save or process your filtered data
        print(f"\nNumber of points within {desired_radius}m radius: {len(filtered_gdf)}")
        #print(filtered_gdf['ST_B10_Celsius'].head())

    return filtered_gdf

#### Defining boundaries and plotting region of interest using GeoJSON

In [3]:
geojson_path = "C:\\LocalOneDrive\\Documents\\Desktop\\MTI\\UHI-Project\\MSE-ES-UHI\\Data\\SG_geojson\\SG.geojson"

geo_data = gpd.read_file(geojson_path)

# Display the matching features
print(geo_data.columns)

Index(['osm_id', 'osm_type', 'addr_street', 'building', 'name', 'access_roof',
       'building_material', 'addr_housenumber', 'roof_material', 'geometry'],
      dtype='object')


In [4]:
blocks_of_interest = ['112', '114', '113', '115']

polygons = {}

for block in blocks_of_interest:
    matching_features = geo_data[(geo_data['addr_street'].str.contains("Bukit Purmei", na=False)) & 
                                (geo_data['addr_housenumber'].str.contains(f"{block}", na=False))]

    if not matching_features.empty:
        polygon = matching_features.iloc[0]['geometry']
        polygons[f'polygon_{block}'] = polygon
        print(polygons[f'polygon_{block}'])
    else:
        print("No matching features found.")

POLYGON ((103.8254384 1.2743995, 103.825484 1.2743955, 103.8254826 1.2743647, 103.8257254 1.274362, 103.8257254 1.2744009, 103.8260325 1.2743915, 103.8260754 1.2744344, 103.8262618 1.274429, 103.8263745 1.2745524, 103.8263785 1.2747656, 103.8262082 1.2747669, 103.8262045 1.274549, 103.8257106 1.2745598, 103.8257079 1.2745108, 103.8254424 1.2745122, 103.8254384 1.2743995))
POLYGON ((103.8254451 1.274657, 103.8255926 1.2746516, 103.8255886 1.2750116, 103.8256382 1.2750605, 103.8258664 1.2750513, 103.8258635 1.2749908, 103.8261706 1.2749835, 103.8261733 1.2750663, 103.82622 1.2750652, 103.8262177 1.2749582, 103.826303 1.2749563, 103.8263065 1.2751152, 103.8263989 1.2751132, 103.8264009 1.2752002, 103.8262434 1.2752037, 103.8262454 1.2752961, 103.8261471 1.2752983, 103.8261437 1.2751449, 103.8258625 1.2751533, 103.8258635 1.2751839, 103.8255618 1.2751933, 103.8254545 1.2750927, 103.8254451 1.274657))
POLYGON ((103.8264643 1.2746563, 103.8268371 1.2746362, 103.8268412 1.2746898, 103.8270133

In [6]:
# Convert the dictionary to a list of dictionaries suitable for creating a GeoDataFrame
data_list = [{'block': key, 'geometry': value} for key, value in polygons.items()]

# Create the GeoDataFrame
bp_polygons = gpd.GeoDataFrame(data_list)

##### Filtering 30m x 30m pixels based on region of interest

##### Using ESPG:3857 allows you to blow up the pixels in metres because the coordinate representation is in metres

In [6]:
# Suppress warnings
logging.getLogger('bokeh').setLevel(logging.ERROR)
pd.options.mode.chained_assignment = None  # default='warn'

global within_polygon_gdf

def plot_spatial_map(filtered_gdf): 
    global within_polygon_gdf
    
    filtered_gdf = filtered_gdf.to_crs('epsg:3857')

    # print(filtered_gdf['geometry'])

    # Create pixels as 30m x 30m boxes around each point
    # Assuming each point is at the center of the pixel
    half_width = 15  # half the width of the pixel in meters since the ESPG:3857 coordinate system is in metres
    filtered_gdf['geometry'] = filtered_gdf['geometry'].apply(lambda x: box(x.x - half_width, x.y - half_width, x.x + half_width, x.y + half_width))

    #print(filtered_gdf['geometry'])

    # Create a GeoDataFrame from all polygons and convert CRS to match
    polygon_gdf = gpd.GeoDataFrame({'geometry': list(polygons.values())}, crs='epsg:4326')
    polygon_gdf_3857 = polygon_gdf.to_crs('epsg:3857')

    # Filter points that intersect any polygon
    def intersects_any_polygon(point):
        return any(point.intersects(poly) for poly in polygon_gdf['geometry'])
    
    filtered_gdf['intersects'] = filtered_gdf['geometry'].apply(intersects_any_polygon)

    # Check intersection with any polygon
    within_polygon_gdf = filtered_gdf[filtered_gdf['intersects']].copy()

    # print(polygon_gdf_3857['geometry'])

    # Filter points that intersect any polygon
    filtered_gdf['intersects'] = filtered_gdf['geometry'].apply(
        lambda geom: any(geom.intersects(poly) for poly in polygon_gdf_3857['geometry']))
    within_polygon_gdf = filtered_gdf[filtered_gdf['intersects']].copy()

    print("Number of pixels in region of interest: " + str(len(within_polygon_gdf)))

    # Print or use the filtered GeoDataFrame as needed
    # print("\nNumber of points within the region of interest: " + str(len(within_polygon_gdf)))

    # # Print the centroids of the intersected pixels
    # for index, row in within_polygon_gdf.iterrows():
    #     centroid = row['geometry'].centroid
    #     print(f"Longitude: {centroid.x}, Latitude: {centroid.y}")

    # Define a function to select a subset of the color palette
    def select_colors(palette, n):
        return [palette[int(i)] for i in np.linspace(0, len(palette)-1, n)]

    # Create a custom color scale using a continuous palette
    custom_palette = select_colors(Inferno256, 256)  # More colors for smoother transitions

    # Create the heatmap using the centroid points of intersected pixels
    heatmap = within_polygon_gdf.hvplot.points('Longitude', 'Latitude', geo=True, c='ST_B10_Celsius', cmap=custom_palette, size=5, tiles='OSM', frame_width=700, frame_height=500, colorbar=True, clim=(20, 40))

    # Plot square polygons with the same color mapping as the points
    squares_plot = within_polygon_gdf.hvplot.polygons('geometry', c='ST_B10_Celsius', cmap=custom_palette, alpha=0.5, colorbar=True, clim=(20, 40))

    # Plot the polygon with visible settings
    polygon_plot = polygon_gdf.hvplot(geo=True, color='red', line_width=3, alpha=0.7)

    # Overlay the polygon onto the heatmap
    overlay_map = polygon_plot * heatmap * squares_plot

    # Set up Panel to display the plot
    # pane = pn.panel(overlay_map)

    # pane.show()
    # pane.save(f'C:\\LocalOneDrive\\Documents\\Desktop\\MTI\\UHI-Project\\MSE-ES-UHI\\MSE-ES-UHI\\2_landsat\\Heatmaps\\{postal_code_112}_{satellite_image}_LST_Filtered.html', embed=True)

    return overlay_map

#### Plotting LST over time

##### Combining GDFs

In [21]:
# Note that Landsat9 only has data from 2021 onwards
year = "2024"

# Suppress warnings
logging.getLogger('bokeh').setLevel(logging.ERROR)
pd.options.mode.chained_assignment = None  # default='warn'

# Specify the zip file and temporary directory for extraction
zip_file_path = f"C:\\LocalOneDrive\\Documents\\Desktop\\MTI\\UHI-Project\\MSE-ES-UHI\\Data\\Landsat9\\{year}.zip"
temp_dir = f"C:\\LocalOneDrive\\Documents\\Desktop\\MTI\\UHI-Project\\MSE-ES-UHI\\Data\\temp_extract"

# Create a temporary directory if it doesn't exist
os.makedirs(temp_dir, exist_ok=True)

# Extract the .tif files from the zip
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(temp_dir)

# Initialize an empty list to hold all the GeoDataFrames
gdfs = []

# Walk through the temporary directory and process each .tif file
for filename in os.listdir(f"{temp_dir}\\{year}"):
    if filename.endswith(".tif"):
        print("Currently processing: " + filename)
        file_path = os.path.join(f"{temp_dir}\\{year}", filename)
        
        # Extract the time period from the filename
        # Assuming filename format is "L8_UTC_YYYYMMDD_hhmmss.tif"
        time_str = filename.split('_')[2]
        time_obj = datetime.strptime(time_str, "%Y%m%d")
        
        # Load and preprocess the GeoDataFrame
        gdf = preprocessing(file_path)
        gdf['time'] = time_obj  # Append the datetime object as a new column
        
        # Append the processed GeoDataFrame to the list
        gdfs.append(gdf)

# Combine all GeoDataFrames into one
combined_gdf = pd.concat(gdfs)

shutil.rmtree(f"C:\\LocalOneDrive\\Documents\\Desktop\\MTI\\UHI-Project\\MSE-ES-UHI\\Data\\temp_extract")

# Use the combined GeoDataFrame as needed
print(combined_gdf)

Currently processing: L9_UTC_20240109_031653.tif
Total number of valid pixels: 80849
   Latitude   Longitude
0  1.470099  103.589751
1  1.470099  103.590021
2  1.470099  103.590290
3  1.470099  103.590560
4  1.470100  103.590830

Number of points within 200m radius: 0
Currently processing: L9_UTC_20240125_031651.tif
Total number of valid pixels: 526061
   Latitude   Longitude
0  1.470099  103.589751
1  1.470099  103.590021
2  1.470099  103.590290
3  1.470099  103.590560
4  1.470100  103.590830

Number of points within 200m radius: 0
Currently processing: L9_UTC_20240210_031656.tif
Total number of valid pixels: 0
   Latitude   Longitude
0  1.470099  103.589751
1  1.470099  103.590021
2  1.470099  103.590290
3  1.470099  103.590560
4  1.470100  103.590830

Number of points within 200m radius: 0
Currently processing: L9_UTC_20240226_031644.tif
Total number of valid pixels: 95904
   Latitude   Longitude
0  1.470099  103.589751
1  1.470099  103.590021
2  1.470099  103.590290
3  1.470099  10

##### Spatial plot over time

In [15]:
# Create an interactive plot with filtering based on the GeoDataFrame
def create_interactive_plot(combined_gdf):
    # Create a list of unique dates sorted
    unique_dates = combined_gdf['time'].dt.strftime('%Y-%m-%d').sort_values().unique()
    # print(f"Unique Dates: {unique_dates}")

    date_index_map = {i + 1: date for i, date in enumerate(unique_dates)}

    # Setup an integer slider to select time periods
    time_slider = pn.widgets.IntSlider(name='Select Time', start=1, end=len(unique_dates), value=1, step=1)

    @pn.depends(time_slider.param.value_throttled)
    def dynamic_map(value):
        selected_date = date_index_map[value]
        selected_datetime = pd.to_datetime(selected_date).date()
        
        # Filter data for the selected time
        filtered_data = combined_gdf[combined_gdf['time'].dt.date == selected_datetime]
        print(f"Displaying plot for " + str(selected_date))
        
        # Call plot_spatial_map for the selected time period
        return plot_spatial_map(filtered_data)

    layout = pn.Column(
        "<br>\nInteractive Land Surface Temperature Map",
        time_slider,
        dynamic_map
    )

    return layout

layout = create_interactive_plot(combined_gdf)
# layout.servable()
pn.serve(layout, show=False, start=True)

Displaying plot for 2024-02-02
Number of pixels in region of interest: 33
Launching server at http://localhost:64845


Displaying plot for 2024-03-05
Number of pixels in region of interest: 33


#### Exporting data to .csv

In [10]:
# import geopandas as gpd
# import pandas as pd
# from shapely.geometry import box

# def filter_and_save_data(year_gdf, polygons, output_file):
#     # Ensure polygons are in EPSG:3857
#     polygon_gdf = gpd.GeoDataFrame({'geometry': list(polygons.values())}, crs='epsg:4326')
#     polygon_gdf = polygon_gdf.to_crs('epsg:3857')

#     # Initialize an empty DataFrame to store all filtered data
#     all_filtered_data = gpd.GeoDataFrame()

#     for date in year_gdf['time'].dt.strftime('%Y-%m-%d').sort_values().unique():
#         # Filter data for the specific date
#         date_data = year_gdf[year_gdf['time'].dt.strftime('%Y-%m-%d') == date]

#         # Convert CRS to EPSG:3857 and create 30m x 30m boxes around each point
#         date_data = date_data.to_crs('epsg:3857')
#         date_data['geometry'] = date_data['geometry'].apply(
#             lambda x: box(x.x - 15, x.y - 15, x.x + 15, x.y + 15))

#         # Filter points that intersect any polygon
#         date_data['intersects'] = date_data['geometry'].apply(
#             lambda geom: any(geom.intersects(poly) for poly in polygon_gdf['geometry']))
#         filtered_data = date_data[date_data['intersects']].copy()

#         # Append the filtered data of this date to the all_filtered_data DataFrame
#         all_filtered_data = pd.concat([all_filtered_data, filtered_data], ignore_index=True)

#     # Drop the 'geometry' column as it cannot be saved directly in CSV format
#     all_filtered_data.drop(columns=['geometry'], inplace=True)

#     # Save the aggregated filtered data to a CSV file
#     all_filtered_data.to_csv(output_file, index=False)
#     print(f"Data successfully exported to {output_file}")

# combined_gdf['time'] = pd.to_datetime(combined_gdf['time'])  # Ensure 'time' is a datetime object
# output_path = 'C:\\LocalOneDrive\\Documents\\Desktop\\MTI\\UHI-Project\\MSE-ES-UHI\\Data\\FilteredData\\BukitPurmei\\BukitPurmei_Filtered_2019.csv'
# filter_and_save_data(combined_gdf, polygons, output_path)

Data successfully exported to C:\LocalOneDrive\Documents\Desktop\MTI\UHI-Project\MSE-ES-UHI\Data\FilteredData\BukitPurmei\BukitPurmei_Filtered_2019.csv


In [22]:
def filter_and_save_data(year_gdf, polygons, output_file):
    # Convert the polygons dictionary to a GeoDataFrame
    polygon_gdf = gpd.GeoDataFrame({
        'block': list(polygons.keys()),   # Keys from your dictionary
        'geometry': list(polygons.values())
    }, crs='epsg:4326')
    polygon_gdf = polygon_gdf.to_crs('epsg:3857')

    # Initialize an empty DataFrame to store all filtered data
    all_filtered_data = gpd.GeoDataFrame()

    for date in year_gdf['time'].dt.strftime('%Y-%m-%d').sort_values().unique():
        # Filter data for the specific date
        date_data = year_gdf[year_gdf['time'].dt.strftime('%Y-%m-%d') == date]

        # Convert CRS to EPSG:3857 and create 30m x 30m boxes around each point
        date_data = date_data.to_crs('epsg:3857')
        date_data['geometry'] = date_data['geometry'].apply(
            lambda x: box(x.x - 15, x.y - 15, x.x + 15, x.y + 15))

        # Determine the block for each point by finding which polygon it intersects
        def find_block(geom):
            for idx, poly in polygon_gdf.iterrows():
                if geom.intersects(poly['geometry']):
                    return poly['block']
            return None  # Return None or an appropriate value if no intersection is found

        date_data['block'] = date_data['geometry'].apply(find_block)

        # Filter to keep only data that intersects with a polygon
        filtered_data = date_data[date_data['block'].notnull()].copy()

        # Append the filtered data of this date to the all_filtered_data DataFrame
        all_filtered_data = pd.concat([all_filtered_data, filtered_data], ignore_index=True)

    # Drop the 'geometry' column as it cannot be saved directly in CSV format
    all_filtered_data.drop(columns=['geometry'], inplace=True)

    print(all_filtered_data)

    # Save the aggregated filtered data to a CSV file
    all_filtered_data.to_csv(output_file, index=False)
    print(f"Data successfully exported to {output_file}")

# Example usage, assuming combined_gdf and polygons are defined earlier
combined_gdf['time'] = pd.to_datetime(combined_gdf['time'])  # Ensure 'time' is a datetime object
output_path = 'C:\\LocalOneDrive\\Documents\\Desktop\\MTI\\UHI-Project\\MSE-ES-UHI\\Data\\FilteredData\\BukitPurmei\\Landsat9\\BukitPurmei_Filtered_2024_Blocks.csv'
filter_and_save_data(combined_gdf, polygons, output_path)

     Longitude  Latitude    SR_B1    SR_B2    SR_B3    SR_B4    SR_B5  \
0   103.826035  1.275400   8198.0   8984.0  10204.0  10463.0  15004.0   
1   103.826304  1.275400   8484.0   8936.0  10554.0   9659.0  16765.0   
2   103.826574  1.275400   8545.0   9406.0  10666.0  11040.0  14674.0   
3   103.826843  1.275400   8602.0   9188.0  10778.0  11020.0  15889.0   
4   103.827113  1.275400   8291.0   8875.0  10309.0  10573.0  14734.0   
..         ...       ...      ...      ...      ...      ...      ...   
61  103.827383  1.274586   8036.0   8942.0  11046.0  11197.0  16314.0   
62  103.825496  1.274314   9662.0  10704.0  12203.0  12712.0  17139.0   
63  103.825765  1.274314  10140.0  11060.0  12246.0  13117.0  17749.0   
64  103.826035  1.274314   9927.0  10359.0  11765.0  12066.0  15698.0   
65  103.826305  1.274314   7707.0   8425.0  10002.0  10136.0  14229.0   

      SR_B6    SR_B7  SR_QA_AEROSOL  ...  ST_ATRAN_Scaled  ST_CDIST_Scaled  \
0   13267.0  10908.0          224.0  ...     

#### Codes to combine data from 2018 to 2024

In [23]:
# Define the base file path
base_path = r"C:\LocalOneDrive\Documents\Desktop\MTI\UHI-Project\MSE-ES-UHI\Data\FilteredData\BukitPurmei\Landsat9"

# File names
files = [
    r"BukitPurmei_Filtered_2021_Blocks.csv",
    r"BukitPurmei_Filtered_2022_Blocks.csv",
    r"BukitPurmei_Filtered_2023_Blocks.csv",
    r"BukitPurmei_Filtered_2024_Blocks.csv"
]

# Read and concatenate the CSV files
df_list = [pd.read_csv(f"{base_path}\\{file_name}") for file_name in files]
combined_df = pd.concat(df_list, ignore_index=True)

# Save the combined DataFrame to a new CSV file
combined_df.to_csv(f"{base_path}\\BukitPurmei_Filtered_2021_to_2024_Blocks.csv", index=False)

print("Files were successfully concatenated and saved.")

Files were successfully concatenated and saved.


## Plot Spatial Map

In [18]:
bp_treatment = ['polygon_112', 'polygon_114']

In [8]:
bp_2023_csv_path = r"C:\LocalOneDrive\Documents\Desktop\MTI\UHI-Project\MSE-ES-UHI\Data\FilteredData\BukitPurmei\Landsat8\BukitPurmei_Filtered_2023_Blocks.csv"

bp_2023_gdf = pd.read_csv(bp_2023_csv_path)

geometry = [Point(xy) for xy in zip(bp_2023_gdf['Longitude'], bp_2023_gdf['Latitude'])]
bp_2023_gdf = gpd.GeoDataFrame(bp_2023_gdf, crs="EPSG:4326", geometry=geometry)

bp_polygons = bp_polygons.set_crs('EPSG:4326')
bp_polygons = bp_polygons.rename(columns={'addr_housenumber': 'BLOCK_NO'})

bp_2023_gdf = gpd.GeoDataFrame(bp_2023_gdf[bp_2023_gdf['time'] == '2023-10-13'])

In [11]:
def plot_spatial_map_layered(bp_gdf, bp_polygons):
    bp_gdf = bp_gdf.to_crs('EPSG:3857')
    bp_gdf['geometry'] = bp_gdf.geometry.apply(lambda p: box(p.x - 15, p.y - 15, p.x + 15, p.y + 15))

    average_temps_by_blk = bp_gdf.groupby(['block'])['ST_B10_Celsius'].mean()

    min_temp = average_temps_by_blk.min()
    max_temp = average_temps_by_blk.max()

    # print(average_temps_by_block)

    bp_polygons = bp_polygons.to_crs('EPSG:4326')

    bp_polygons.reset_index(drop=True, inplace=True)
    
    average_temps_by_blk.rename('Average_Temp', inplace=True)  

    bp_polygons = bp_polygons.join(average_temps_by_blk, on='block')

    bp_polygons['Average_Temp'].fillna(0, inplace=True)

    # print(all_polygons)

    pixels_plot = bp_gdf.hvplot.points('Longitude', 'Latitude', geo=True, c='ST_B10_Celsius', cmap=Oranges[256][::-1], frame_width=700, frame_height=500, colorbar=True, clim=(min_temp, max_temp), tiles='OSM', size=10)
    squares_plot = bp_gdf.hvplot.polygons('geometry', c='ST_B10_Celsius', cmap=Oranges[256][::-1], alpha=0.5, frame_width=700, frame_height=500, colorbar=True, clim=(min_temp, max_temp))
    block_plot = bp_polygons.hvplot(
        geo=True,
        c='Average_Temp',
        cmap=Oranges[256][::-1],
        clim=(min_temp, max_temp),
        hover_cols=['Average_Temp'],
        tiles='OSM',
        colorbar=True,
        frame_width=500,
        frame_height=450,
        line_color='black'
    )

    combined_plot = block_plot * squares_plot * pixels_plot

    return combined_plot

In [19]:
bp_2023_gdf_treatment = bp_2023_gdf[bp_2023_gdf['block'].isin(bp_treatment)]
bp_polygons_treatment = bp_polygons[bp_polygons['block'].isin(bp_treatment)]

In [22]:
plot_spatial_map_layered(bp_2023_gdf_treatment, bp_polygons_treatment)

:Overlay
   .WMTS.I      :WMTS   [Longitude,Latitude]
   .Polygons.I  :Polygons   [Longitude,Latitude]   (Average_Temp)
   .Polygons.II :Polygons   [x,y]   (ST_B10_Celsius)
   .WMTS.II     :WMTS   [Longitude,Latitude]
   .Points.I    :Points   [Longitude,Latitude]   (ST_B10_Celsius)

In [24]:
bp_2023_gdf_control = bp_2023_gdf[~bp_2023_gdf['block'].isin(bp_treatment)]
bp_polygons_control = bp_polygons[~bp_polygons['block'].isin(bp_treatment)]

In [25]:
plot_spatial_map_layered(bp_2023_gdf_control, bp_polygons_control)

:Overlay
   .WMTS.I      :WMTS   [Longitude,Latitude]
   .Polygons.I  :Polygons   [Longitude,Latitude]   (Average_Temp)
   .Polygons.II :Polygons   [x,y]   (ST_B10_Celsius)
   .WMTS.II     :WMTS   [Longitude,Latitude]
   .Points.I    :Points   [Longitude,Latitude]   (ST_B10_Celsius)